In [1]:
# ============================================================
# Deliverable 2: Regression Modeling & Evaluation
# Dataset: Online Retail II (Excel)
# Goal: Predict transaction value (log-transformed revenue)
# ============================================================

# ----------------------------
# Core data libraries
# ----------------------------
import pandas as pd
import numpy as np

# ----------------------------
# Visualization libraries
# ----------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# ----------------------------
# Scikit-learn (ML)
# ----------------------------
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score


In [2]:
# ============================================================
# Step 1: Load the dataset
# ============================================================

df = pd.read_excel("online_retail_II.xlsx")

print("Raw dataset shape (rows, cols):", df.shape)
print("\nColumn names:")
print(list(df.columns))

display(df.head())

Raw dataset shape (rows, cols): (525461, 8)

Column names:
['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country']


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [3]:
# ============================================================
# Step 2: Standardize column names
# - removes whitespace issues that break column selection
# ============================================================

df.columns = df.columns.str.strip().str.replace(" ", "")

print("Updated columns:")
print(list(df.columns))


Updated columns:
['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'CustomerID', 'Country']


In [4]:
# ============================================================
# Step 3: Data Cleaning
# Why cleaning is needed:
# - CustomerID missing -> cannot track customers
# - Description missing -> product becomes unclear
# - Duplicates -> double count revenue
# - Negative quantity -> returns/cancellations
# - Non-positive price -> invalid transactions
# ============================================================

# Remove missing critical values
df = df.dropna(subset=["CustomerID", "Description"])

# Remove exact duplicate rows
df = df.drop_duplicates()

# Remove return/cancellation rows (Quantity <= 0)
df = df[df["Quantity"] > 0]

# Rename Price to UnitPrice (some versions of dataset use "Price")
if "Price" in df.columns:
    df = df.rename(columns={"Price": "UnitPrice"})

# Remove invalid pricing
df = df[df["UnitPrice"] > 0]

# Convert InvoiceDate to datetime; drop rows that fail conversion
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")
df = df.dropna(subset=["InvoiceDate"])

print("Cleaned dataset shape:", df.shape)
display(df.head())


Cleaned dataset shape: (400916, 8)


,Invoice,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [5]:
# ============================================================
# Step 4: Feature Engineering (Required for Deliverable 2)
# Create features that help regression learn patterns
# ============================================================

# Transaction-level revenue
df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]

# Time-based features (often useful for retail behavior)
df["Year"] = df["InvoiceDate"].dt.year
df["Month"] = df["InvoiceDate"].dt.month
df["DayOfWeek"] = df["InvoiceDate"].dt.dayofweek  # Monday=0, Sunday=6
df["Hour"] = df["InvoiceDate"].dt.hour

# Log transform reduces extreme skew in revenue (common in retail datasets)
df["LogTotalPrice"] = np.log1p(df["TotalPrice"])

print("Feature engineering completed. Preview:")
display(df[["Quantity", "UnitPrice", "TotalPrice", "LogTotalPrice", "Month", "DayOfWeek", "Hour"]].head())


Feature engineering completed. Preview:


,Quantity,UnitPrice,TotalPrice,LogTotalPrice,Month,DayOfWeek,Hour
0,12,6.95,83.4,4.435567,12,1,7
1,12,6.75,81.0,4.406719,12,1,7
2,12,6.75,81.0,4.406719,12,1,7
3,48,2.10,100.8,4.623010,12,1,7
4,24,1.25,30.0,3.433987,12,1,7


In [6]:
# ============================================================
# Step 6: Additional Feature Engineering (Feature Set B)
# Add invoice-level behavioral features
# ============================================================

# Number of unique products in each invoice (basket diversity)
df["InvoiceUniqueItems"] = df.groupby("Invoice")["StockCode"].transform("nunique")

# Total quantity of items purchased in that invoice (basket size)
df["InvoiceTotalQuantity"] = df.groupby("Invoice")["Quantity"].transform("sum")

print("Feature Set B added:")
display(df[["Invoice", "StockCode", "Quantity", "InvoiceUniqueItems", "InvoiceTotalQuantity"]].head())


Feature Set B added:


,Invoice,StockCode,Quantity,InvoiceUniqueItems,InvoiceTotalQuantity
0,489434,85048,12,8,166
1,489434,79323P,12,8,166
2,489434,79323W,12,8,166
3,489434,22041,48,8,166
4,489434,21232,24,8,166


In [7]:
# ============================================================
# Step 7: Define target (y) and feature matrix (X)
# ============================================================

# Target variable
y = df["LogTotalPrice"]

# Drop columns we should NOT use for prediction
X = df.drop(columns=["TotalPrice", "LogTotalPrice", "InvoiceDate"])

# Remove free-text column (not useful for regression baseline)
if "Description" in X.columns:
    X = X.drop(columns=["Description"])

print("Feature matrix shape:", X.shape)
display(X.head())


Feature matrix shape: (400916, 12)


,Invoice,StockCode,Quantity,UnitPrice,CustomerID,Country,Year,Month,DayOfWeek,Hour,InvoiceUniqueItems,InvoiceTotalQuantity
0,489434,85048,12,6.95,13085.0,United Kingdom,2009,12,1,7,8,166
1,489434,79323P,12,6.75,13085.0,United Kingdom,2009,12,1,7,8,166
2,489434,79323W,12,6.75,13085.0,United Kingdom,2009,12,1,7,8,166
3,489434,22041,48,2.10,13085.0,United Kingdom,2009,12,1,7,8,166
4,489434,21232,24,1.25,13085.0,United Kingdom,2009,12,1,7,8,166


In [8]:
# ============================================================
# Step 8: Fix categorical column data types
# Prevents OneHotEncoder errors (mixed int/str issue)
# ============================================================

# Identify numeric vs categorical columns
num_cols = X.select_dtypes(include="number").columns.tolist()
cat_cols = X.select_dtypes(exclude="number").columns.tolist()

# Convert all categorical columns to string
X = X.copy()
for c in cat_cols:
    X[c] = X[c].astype(str)

print("Numeric columns:", num_cols)
print("Categorical columns:", cat_cols)
print("Converted categorical columns to string.")


Numeric columns: ['Quantity', 'UnitPrice', 'CustomerID', 'Year', 'Month', 'DayOfWeek', 'Hour', 'InvoiceUniqueItems', 'InvoiceTotalQuantity']
Categorical columns: ['Invoice', 'StockCode', 'Country']
Converted categorical columns to string.


In [9]:
# ============================================================
# Step 9: Split data into training and testing sets
# ============================================================

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Training size:", X_train.shape)
print("Testing size :", X_test.shape)

Training size: (320732, 12)
Testing size : (80184, 12)


In [10]:
# ============================================================
# Step 10: Preprocessing Pipeline
# ============================================================

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
    ]
)

print("Preprocessing pipeline ready.")


Preprocessing pipeline ready.


In [11]:
# ============================================================
# Step 11: Linear Regression Model
# ============================================================

from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

lr_model = Pipeline(steps=[
    ("prep", preprocess),
    ("model", LinearRegression())
])

lr_model.fit(X_train, y_train)

pred_lr = lr_model.predict(X_test)

mse_lr = mean_squared_error(y_test, pred_lr)
rmse_lr = np.sqrt(mse_lr)
r2_lr = r2_score(y_test, pred_lr)

print("Linear Regression Results")
print("RMSE:", rmse_lr)
print("R^2 :", r2_lr)


Linear Regression Results
RMSE: 0.5248161921755014
R^2 : 0.7242334249959191


In [12]:
# ============================================================
# Step 12: Ridge Regression Model
# ============================================================

from sklearn.linear_model import Ridge

ridge_model = Pipeline(steps=[
    ("prep", preprocess),
    ("model", Ridge(alpha=1.0))
])

ridge_model.fit(X_train, y_train)

pred_ridge = ridge_model.predict(X_test)

mse_ridge = mean_squared_error(y_test, pred_ridge)
rmse_ridge = np.sqrt(mse_ridge)
r2_ridge = r2_score(y_test, pred_ridge)

print("Ridge Regression Results")
print("RMSE:", rmse_ridge)
print("R^2 :", r2_ridge)


Ridge Regression Results
RMSE: 0.5235075633081582
R^2 : 0.7256069579924218


In [13]:
# ============================================================
# Step 13: Cross Validation
# ============================================================

from sklearn.model_selection import cross_val_score

cv_lr = cross_val_score(
    lr_model, X, y, cv=5,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

cv_ridge = cross_val_score(
    ridge_model, X, y, cv=5,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

cv_rmse_lr = np.sqrt(-cv_lr.mean())
cv_rmse_ridge = np.sqrt(-cv_ridge.mean())

print("Cross Validation RMSE")
print("Linear:", cv_rmse_lr)
print("Ridge :", cv_rmse_ridge)


Cross Validation RMSE
Linear: 0.8801039110066392
Ridge : 0.8546108923783924


In [14]:
results = pd.DataFrame({
    "Model": ["Linear Regression", "Ridge Regression"],
    "Test RMSE": [rmse_lr, rmse_ridge],
    "Test R^2": [r2_lr, r2_ridge],
    "CV RMSE": [cv_rmse_lr, cv_rmse_ridge]
})

display(results)

,Model,Test RMSE,Test R^2,CV RMSE
0,Linear Regression,0.524816,0.724233,0.880104
1,Ridge Regression,0.523508,0.725607,0.854611
